In [1]:
import numpy as np
from PIL import Image


def codPred(nombreI, nombreS, m):
    # Cargar la imagen y convertirla a escala de grises
    img = Image.open(nombreI).convert("L")
    x = np.array(img, dtype=np.float64)

    # Obtener las dimensiones de la imagen
    nFilas, nColumnas = x.shape

    # Abrir el archivo para escribir los datos
    with open(nombreS, "wb") as f:
        # Escribir las dimensiones y el valor m
        f.write(np.uint16(nFilas).tobytes())
        f.write(np.uint16(nColumnas).tobytes())
        f.write(np.uint8(m).tobytes())

        # Convertir la imagen a un arreglo 1D (Raster)
        x1D = x.T.flatten()

        # Escribir el primer valor de x1D
        f.write(np.uint8(x1D[0]).tobytes())

        nBits = 8  # Inicializamos el contador de bits

        # Procesar los píxeles restantes
        for iPixel in range(1, nFilas * nColumnas):
            e = x1D[iPixel] - x1D[iPixel - 1]
            if e <= 0:
                ep = abs(e) * 2 - 1
            else:
                ep = e * 2

            eq = int(np.floor(ep / 2**m))  # Parte entera
            er = int(np.remainder(ep, 2**m))  # Parte restante

            # Crear el código UNARIO
            cq = np.zeros(eq, dtype=np.uint8)
            cr = np.array([int(x) for x in format(er, f"0{m}b")], dtype=np.uint8)

            # Concatenar los códigos
            c = np.concatenate((cq, cr))

            # Escribir los bits en el archivo
            f.write(c.tobytes())
            nBits += len(c)

        nBits += 35  # Ajustar el contador de bits
        R = nBits / (nFilas * nColumnas)

    return nBits, R